In [9]:
# ================ Demand System ================
mutable struct DemandSystem
    Nodes # 2N length vector of xy-coords per node
    F     # N length vector of demand per node

    function DemandSystem(Nodes_in, F_in)
        new(Nodes_in, F_in)
    end
end

# --- Gets the coordinates of the ith node ---
function nCrd(M::DemandSystem, i)
    return M.Nodes[Int(2*i-1)], M.Nodes[Int(2*i)]
end

# --- Gets the demand of the ith node ---
function nDmd(M::DemandSystem, i)
    return M.F[Int(i)]
end


# ================ Design Variable ================
# --- Gets the coordinates of the kth station ---
function sCrd(x, k)
    return x[Int(4*k)-3], x[Int(4*k)-2]
end

# --- Gets the capacity of the kth station ---
function sCap(x, k)
    return x[Int(4*k)-1]
end

# --- Gets the building boolean of the kth station ---
function sB(x, k)
    return x[Int(4*k)]
end


# ================ Objective Function ================
# --- OBJECTIVE FUNCTION: Total cost of all stations ---
function cost(x, c, f)
    sz, sb = NaN, NaN
    sum = 0

    M = floor(length(x)/4)
    for k = 1:M
        sz = sCap(x, k)
        sb = sB(x, k)

        sum += (c + f*sz)*sb
    end
    return sum
end


# ================ Constraint Functions ================
# --- Squared distance between a node and a station ---
function i_kDist(nu, nv, sx, sy)
    return (nu-sx)*(nu-sx) + (nv-sy)*(nv-sy)
end

# --- INEQUALITY CONSTRAINT i: The ith node's demand is met by the stations ---
#       (  There are N constraints of this form. One for each i = 1,...,N  )
#       (  In the form of g_i(x) <= 0  )
function nodeConstraint_i(M::DemandSystem, x, i)
    nx, ny = nCrd(M, i)
    F = nDmd(M, i)
    sum = 0

    M = length(x)/4
    for k = 1:M
        sx, sy = sCrd(x, k)
        sz     = sCap(x, k)
        sb     = sB(x, k)
        dist = i_kDist(nx, ny, sx, sy)

        sum += (sz/dist)*sb
    end

    return F - sum
end 

# --- EQUALITY CONSTRAINT k: The kth station's b_k is a boolean --
#       (  There are M constraints of this form. One for each k = 1,...,M  )
#       (  In the form of h_k(x) = 0  )
function stationConstraint_k(x, k)
    sb = sB(x, k)
    return sb*sb - sb
end


# ================ Reading Data ================
# --- Takes a design variable and interprates it into meaningful data ---
#       (  Returns list of coordinates and capacity for built stations only  )
#       (  Returns [ (x,y,z), ..., (x,y,z) ] for built stations only  )
function readVariable(x)
    TOL = 1e-3
    
    result = []
    M = length(x)/4
    for k = 1:M
        sx, sy = sCrd(x, k)
        sz = sCap(x, k)
        sb = sB(x, k)

        if (abs(sb-1) <= TOL)
            push!(result, (sx, sy, sz))
        end
    end

    return result
end

readVariable (generic function with 1 method)

In [2]:

function simulated_annealing(f, x, T, t, k_max) 
    # optimization method used = Simulated Annealing
    # source(s) of code = Kochenderfer & Wheeler Textbook, Algorithm 8.4, page 130
    y = f(x)
    x_best, y_best = x, y
    count = 1
    convergence = -1
    for k in 1 : k_max
        x′ = x .+ rand(T)
        for i in 1:T/4
            x′[Int(4*i)] = rand(0:1)
#             x′[Int(4*i)] = 1
            x'[Int(4*i-1)] = round(x'[Int(4*i-1)])
        end
        y′ = f(x′)
        Δy = y′ - y
        convergence = abs(Δy)     #compute convergence measure
        if Δy ≤ 0 || rand() < exp(-Δy/t(k))
            x, y = x′, y′ 
        end
        if y′ < y_best
            x_best, y_best = x′, y′ 
        end
        count += 1     # update number of function evaluations
    end
    return x_best, count, convergence
end

# Temperature scheduling (exponential annealing scheduling)
𝛄 = 0.9
t(k) = t0*(𝛄^(k-1))
t0 = 10

10

In [3]:

function simple_penalty(x, M, max_station)
    # simple penalty function: counts number of constraint equations violated
    gi_violated = 0
    hk_violated = 0
    for i in 1:length(x)/4
        nodeConstraint_i(M, x, i)>0 ? gi_violated+=1 : continue
    end
        
    return gi_violated 
end

function quadratic_penalty(x, M, max_station)
    # quadratic penalty function
    gi_violated = 0
    hk_violated = 0
    for i in 1:length(x)/4
        gi_violated += max(nodeConstraint_i(M, x, i),0)^2
    end
        
    return gi_violated
end


function penalty_method(f, p1, p2, x, k_max, c, f_cost, M, max_station; ρ1=3, ρ2=2, γ=3, max_iter=2000) 
    # source of code: Kochenderfer & Wheeler Textbook Algorithm 10.1, page 181
    convergence = -1
    x_new = x
    start = time()
    num_eval = 0
    for k in 1 : k_max
        
        x_new, count = simulated_annealing(x -> f(x, c, f_cost) + ρ1*p1(x, M, max_station) + ρ2*p2(x, M, max_station), x, T, t, max_iter) 
        num_eval += count
        ρ1 *= γ
        ρ2 *= γ
        convergence = f(x_new, c, f_cost) - f(x, c, f_cost) 
        if p1(x_new, M, max_station)+p2(x_new, M, max_station) == 0
            return x_new, num_eval, time()-start, convergence 
        end
        x = x_new
    end
    return x, num_eval, time()-start, abs(convergence)
end

penalty_method (generic function with 1 method)

In [4]:
# ================ TEST MODEL ================
using Distributions
using LinearAlgebra
Nodes = [
    1.0, 1.0, 
    1.0, 2.0, 
    2.0, 1.0, 
    2.0, 2.0];

F     = [4, 4, 4, 4];
c, f_cost = 10, 5

sys = DemandSystem(Nodes, F)

x = [
    0, 0, 2, 1,
    0, 0, 2, 1
]
T=8


8

In [10]:
k_max = 1000
x, num_eval, comp_time, convergence = penalty_method(cost, simple_penalty, quadratic_penalty, 
                                                x, k_max, c, f_cost, sys, max_station)

([0.6998858616706974, 0.1371405775667448, 3.0, 0.0, 0.740621034955633, 1.589913549634368, 3.0, 1.0], 2001, 0.5003480911254883, -15.0)

In [11]:
# x = [1,2,3,4,5,6,7,8]
# c = 5
# f_cost = 20
k_max = 3000
# N = 4
max_station = 2
x, num_eval, comp_time, convergence = penalty_method(cost, simple_penalty, quadratic_penalty, x, k_max, c, f_cost, sys, max_station)
# Int(3.0)

([0.6998858616706974, 0.1371405775667448, 3.0, 0.0, 0.740621034955633, 1.589913549634368, 3.0, 1.0], 2001, 0.09184098243713379, 0.0)

In [14]:
println(cost(x,c,f_cost))
println(nodeConstraint_i(sys, x, 1))
println(nodeConstraint_i(sys, x, 2))

25.0
-3.224120873501426
-8.741648318494724


In [15]:
using CSV 
 
Nodes = Array{Float64}(undef, 144)
F = Array{Float64}(undef, 72)

# reading the csv file 
csv_reader = CSV.File("/Users/briannagautama/Downloads/CS268_EVDemand_xy_normalized.csv")

count = 1
for row in csv_reader
    Nodes[2*count-1] = row.x
    Nodes[2*count] = row.y
    F[count] = row.EV_Demand
    count += 1
end